In [1]:
%matplotlib notebook
import numpy as np
import ipywidgets as widgets
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
from matplotlib.widgets import Button
import matplotlib.lines as mlines

In [58]:
class EnvironmentState:
    REGION_WIDTH = 2
    REGION_HEIGHT = 2
    AGENT_X = 0.9
    
    def __init__(self):
        self._agent_state_index = (0,0)
        self._agent_position_x = self.AGENT_X
        self._agent_position_y = 0
        self._agent_aiming_angle = 180
        self._target_center_x = 0
        self._target_center_y = 0
        self._target_height = self.REGION_HEIGHT/10
        
    def randomize(self, seed=None):
        self.draw()
    
    def action_wait(self):
        print('wait')
        self.draw()
    
    def action_pos_up(self):
        self._agent_position_y = self._agent_position_y + 0.05
        self.draw()
    
    def action_pos_down(self):
        self._agent_position_y = self._agent_position_y - 0.05
        self.draw()
    
    def action_aim_up(self):
        self._agent_aiming_angle = self._agent_aiming_angle - 3
        self.draw()
    
    def action_aim_down(self):
        self._agent_aiming_angle = self._agent_aiming_angle + 3
        self.draw()
    
    def draw(self, draw_aiming_ray=False):
        ax = plt.gca()
        ax.clear()
        plt.title('Visualization')
        ax.set_xbound([-self.REGION_WIDTH/2,  self.REGION_WIDTH/2])
        ax.set_ybound([-self.REGION_HEIGHT/2, self.REGION_HEIGHT/2])
         
        agent = plt.Circle(
            (self._agent_position_x, self._agent_position_y), 
            self.REGION_WIDTH/50, 
            color='r'
        )
        ax.add_artist(agent)
        
        span = np.sqrt(self.REGION_WIDTH^2 + self.REGION_HEIGHT^2)
        rx = 10*span*np.cos(np.deg2rad(self._agent_aiming_angle))
        ry = 10*span*np.sin(np.deg2rad(self._agent_aiming_angle))
        l = mlines.Line2D(
            [self._agent_position_x, self._agent_position_x + rx],
            [self._agent_position_y, self._agent_position_y + ry]
        )
        ax.add_line(l)
        
        l = mlines.Line2D(
            [self._target_center_x, self._target_center_x],
            [self._target_center_y-self._target_height/2, self._target_center_y+self._target_height/2]
        )
        ax.add_line(l)
        
        
        

In [59]:
env = EnvironmentState()

apup   = widgets.Button(description='Pos+')
apdown = widgets.Button(description='Pos-')
aaup   = widgets.Button(description='Aim+')
aadown = widgets.Button(description='Aim-')

canvas = widgets.Output()
buttons = widgets.VBox(children=[apup, apdown, aaup, aadown])
all_widgets = widgets.HBox(children=[buttons, canvas])
display(all_widgets)

apup.on_click(lambda x: env.action_pos_up())
apdown.on_click(lambda x: env.action_pos_down())
aaup.on_click(lambda x: env.action_aim_up())
aadown.on_click(lambda x: env.action_aim_down())

with canvas:
    plt.figure(figsize=(4,4))
    env.draw()
    plt.show()

In [ ]:
np.rad2deg